# Chapter 5   SVD and Projections

In [ ]:
# numerical and scientific computing libraries 
import numpy as np
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [46]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

Define a function returning a compact SVD

In [47]:
def SVD_compact(A: np.ndarray, eps) -> np.ndarray:
    # perform QR decomposition on the matrix and return the Q matrix and the R matrix.
    Uf, sf, Vfh = np.linalg.svd(A, full_matrices=False)
    Vf = Vfh.T

    # calculate the rank of the matrix
    rank_A = (np.abs(sf) > eps).sum()
    print("Rank of A:", rank_A)

    # reconstruct the matrix using the SVD
    U = Uf[:, :rank_A]
    s = sf[:rank_A]
    V = Vf[:, :rank_A]

    return U, s, V.T

We also need a $QR$-decomposition.

In [48]:
def classical_QR(A):
    m, n = A.shape

    R = np.zeros((n,n))
    R[0,0] = np.linalg.norm(A[:,0])

    Q = (1/R[0,0])*A[:,0]

    # Gram-Schmidt process
    for j in range(1,n):
        Q = np.column_stack((Q,A[:,j]))
        for i in range(j):
            R[i,j] = np.dot(Q[:,i].T,A[:,j])
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(Q[:,j])
        if np.abs(R[j,j]) < 1e-10:
            raise ValueError("QR factorization failed: A is rank deficient")  
    
        Q[:,j] = (1/R[j,j])*Q[:,j]

    return Q, R

### Projection Representations

Let us confirm the two representations of projections.

In [49]:
# set the dimension
n = 10

### 1. Projection onto a subspace spanned by a single vector

In [50]:
a = np.random.randn(n).reshape(-1,1)
A = a

U, s, Vh = SVD_compact(A, 1e-10)
V = Vh.T

# pseudoinverse from SVD
Pinv = V @ np.diag(1/s) @ U.T
# vector representation of the projection
P = (1/np.linalg.norm(a)**2)* a @ a.T

np.allclose( A@Pinv, P)


Rank of A: 1


True

### 2. Projection onto a subspace spanned by independent vectors 

In [51]:
# set the dimension of the subspace m less than n.
m = 5

# create a random matrix of size n x m who rank is believed to be m.
A = np.random.randn(n, m)

#### Case 1: Orthogonal Vectors

In [52]:
# extract orthogonal vectors from the columns of A
Q, R = classical_QR(A)


# perform the SVD on the matrix A
U, s, Vh = SVD_compact(Q, 1e-10)
V = Vh.T

# pseudoinverse from SVD
Pinv = V @ np.diag(1/s) @ U.T

# projection matrix from orthogonal vectors
P = Q @ Q.T

# check if the pseudoinverse and the projection matrix are the same
print(np.allclose(Pinv, Q.T), np.allclose(Q @ Pinv, P))

Rank of A: 5
True True


#### Case 2: Non-orthogonal independent Vectors

In [53]:
# perform the SVD on the matrix A
U, s, Vh = SVD_compact(A, 1e-10)
V = Vh.T

# pseudoinverse from SVD
Pinv = V @ np.diag(1/s) @ U.T

# keep this pseudoinver for later
Pinv_ind = Pinv

# projection matrix from orthogonal vectors
P = A @ np.linalg.inv(A.T @ A) @ A.T

# check if the pseudoinverse and the projection matrix are the same
print(np.allclose(Pinv, np.linalg.inv(A.T @ A) @ A.T), np.allclose(A @ Pinv, P))

Rank of A: 5
True True


### 3. Projection onto arbitrary vectors

We need proprocessing to create random dependent vectors. We add three more dependent columns and then shuffle the columns.

In [54]:
x = np.random.randn(m,3)
B = np.column_stack((A,A@x))
p = np.random.permutation(m+3)
B = B[:,p]

Now we have no primitive representation of projection without a help of SVD. Also we compare the subspaces spanned by the original vectors and a set of vectors with three more dependent vectors.

In [57]:
# perform SVD on the matrix B
U, s, Vh = SVD_compact(B, 1e-10)
V = Vh.T 

# pseudoinverse from SVD
Pinv = V @ np.diag(1/s) @ U.T  

# check if the pseudoinverse and the projection matrix are the same
print(np.allclose(B @ Pinv, A @ Pinv_ind))
print(B.shape, Pinv.shape, A.shape, Pinv_ind.shape)

Rank of A: 5
True
(10, 8) (8, 10) (10, 5) (5, 10)


Be sure that Pinv and Pinv_ind has different number of rows.